In [1]:
# Some useful settings for interactive work
%load_ext autoreload
%autoreload 2

%matplotlib widget

In [2]:
# Importing the necessary libraries
import figs.render.capture_generation as pg
import figs.visualize.plot_trajectories as pt
import figs.visualize.generate_videos as gv
import figs.scene_editing.scene_editing_utils as scdt

from figs.simulator import Simulator
from figs.control.vehicle_rate_mpc import VehicleRateMPC

import os
os.environ["ACADOS_SOURCE_DIR"] = "/data/madang/FiGS-standalone/acados"
os.environ["LD_LIBRARY_PATH"] = os.getenv("LD_LIBRARY_PATH", "") + "/data/madang/FiGS-standalone/acados/lib"

# print(os.getenv("ACADOS_SOURCE_DIR"))
# print(os.getenv("LD_LIBRARY_PATH"))

# import ctypes
# ctypes.CDLL("/data/<username>/FiGS-standalone/acados/lib/libqpOASES_e.so.3.1")
# ctypes.CDLL("/data/<username>/FiGS-standalone/acados/lib/libblasfeo.so")
# ctypes.CDLL("/data/<username>/FiGS-standalone/acados/lib/libhpipm.so")
# ctypes.CDLL("/data/<username>/FiGS-standalone/acados/lib/libacados.so")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/data/madang/StanfordMSL/FiGS-Standalone/gemsplat/gemsplat/clip/clip.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import packaging


In [3]:
# FiGS Capture Examples (scene_name, capture_name)
capture_examples = [
    # 'backroom'
    # 'sv_1007_gemsplat'
    'packardpark'
]

# FiGS Simulate Examples (scene_name, rollout_name, frame_name, policy_name, course_name)
simulate_examples = [
    # ('flightroom', 'baseline', 'carl', 'vrmpc_fr', 'extended_traj_track'),
    # ('backroom',   'baseline', 'carl', 'vrmpc_fr', 'cluttered_env_track'),
    # ('mid_gate',   'baseline', 'carl', 'vrmpc_fr', 'robustness_track'),
    ('packardpark',   'baseline', 'carl', 'vrmpc_rrt', 'track_spiral')
    # ('sv_917_3_left_gemsplat', 'baseline', 'carl', 'vrmpc_rrt', 'inward_spiral'),
    # ('sv_1007_gemsplat', 'baseline', 'carl', 'vrmpc_fr', 'robustness_track'),
]

# query = 'ladder'

In [4]:
import numpy as np

# Simulate within the FiGS environment
for scene, rollout, frame, policy, course in simulate_examples:
    print("=============================================================")
    print(f"Simulating {scene} scene with {course} course")
    print("-------------------------------------------------------------")

    # Load the policy and simulator
    sim = Simulator(scene,rollout,frame)
    ctl = VehicleRateMPC(course,policy,frame)

    # Use the ideal trajectory in VehicleRateMPC to get initial conditions and final time
    t0,tf,x0 = ctl.tXUd[0,0],ctl.tXUd[0,-1],ctl.tXUd[1:11,0]

    # Simulate the policy
    Tro,Xro,Uro,Imgs,_,_ = sim.simulate(ctl,t0,tf,x0)

    # Output the results
    gv.images_to_mp4(Imgs["rgb"],'test_space/'+course+'_'+scene+'.mp4', ctl.hz)       # Save the video
    # pt.plot_RO_spatial((Tro,Xro,Uro))                               # Plot the spatial trajectory

    # scdt.plot_point_cloud(sim,
    #                       (Tro,Xro,Uro),
    #                       50)

    # Clear the memory of the policy to avoid improper re-initialization of ACADOS
    del ctl

Simulating packardpark scene with track_spiral course
-------------------------------------------------------------


ValueError: The search path '/data/madang/StanfordMSL/3dgs/workspace/outputs/packardpark' did not return any configurations.